In [ ]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

bucket = "racing"
org = "b4mad"
token="PFmq_uLsJ8NZXYuq9zBbNsCbxuerXIapE4N_2kjLzyWauLyZqbscrEJRJw25upSJ1-tKJQAJa8GfItx7Sl4SOw=="
url="https://telemetry.b4mad.racing/"

client = influxdb_client.InfluxDBClient(
   url=url,
   token=token,
   org=org
)

query_api = client.query_api()

query = ' from(bucket: "racing") \
  |> range(start: 2022-05-08T11:43:37.251Z, stop:2022-05-08T12:30:39.231Z) \
  |> filter(fn: (r) => r["_measurement"] == "laps") \
  |> filter(fn: (r) => r["SessionId"] == "89db51de-22a6-4033-8201-2fc37a5fe905") \
  |> filter(fn: (r) => (r["_field"] == "Brake" or r["_field"] == "Clutch" or r["_field"] == "Throttle")) \
  |> aggregateWindow(every: 2s, fn: mean, createEmpty: false) \
  |> yield(name: "mean") ' 

result = query_api.query(org=org, query=query)

results = []
for table in result:
  for record in table.records:
    results.append((record.get_field(), record.get_value()))

print(results)